## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report,accuracy_score
from skimage.io import imread, imshow
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, InputLayer, BatchNormalization, Dense, Dropout, MaxPooling2D, Activation
from keras.optimizers import Adam
import os 

print("All imports done")

Using TensorFlow backend.


All imports done


## Data preprocessing


In [2]:
# First we set the path to find the images
basedir = './dataset'
images_dir = os.path.join(basedir,'image')
labels_filename = 'label.csv'

# This function will loop through the csv file containing the labels and create a dictionnary containing the 
# name of the image as a key and the associate label as a value
def categorical_labelling(): 
    labels_file = open(os.path.join(basedir, labels_filename), 'r')
    lines = labels_file.readlines()
    tumor_labels = {line.split(',')[0] : (line.split(',')[1].strip()) for line in lines[1:]}

    for i in tumor_labels: 
        if tumor_labels[i] == 'no_tumor': 
            tumor_labels[i] = 0
        elif tumor_labels[i] == "meningioma_tumor":
            tumor_labels[i] = 1
        elif tumor_labels[i] == "glioma_tumor":
            tumor_labels[i] = 2
        elif tumor_labels[i] == "pituitary_tumor":
            tumor_labels[i] = 3
    return(tumor_labels)


In [3]:
# This function will loop through the images and using the name of the image and the dictionnary set in the previous 
#function, will return two array containing the features and the label
def extract_features():
    all_features = []
    all_labels = []
    
    labels = categorical_labelling()

    image_paths = [os.path.join(images_dir, l) for l in os.listdir(images_dir)]
    if os.path.isdir(images_dir):
        all_features = []
        all_labels = []
        for img_path in image_paths:
            filename = img_path.split('/')[-1]
            features = imread(img_path, as_gray=True)
            features.shape, features
            all_features.append(features)
            all_labels.append(labels[filename])
    np_features = np.array(all_features)
    np_labels = np.array(all_labels)
    
    np_features =np_features.reshape(-1,512, 512,1) # array of features needs to be changed to correspond to the  
                                                    # input shape of CNN

    
    
    return np_features, np_labels

In [4]:
# This function is used to separate the dataset into a training and testing set with proportion 80% - 20% and returns
# the associated training features, training labels, validation features and validation labels

def get_data(): 

    X, Y = extract_features()

    tr_X = X[:2400]
    tr_Y = Y[:2400]
    te_X = X[2400:]
    te_Y = Y[2400:]
    
    return tr_X, tr_Y, te_X, te_Y

## Testing data preprocessing

In [5]:
## Preparing testing dataset by modifying the function used earlier

testdir = './test'
test_images_dir = os.path.join(testdir,'image')
test_labels_filename = 'label.csv'

def categorical_labelling_testset():
    labels_file = open(os.path.join(testdir, test_labels_filename), 'r')
    lines = labels_file.readlines()
    tumor_labels = {line.split(',')[0] : (line.split(',')[1].strip()) for line in lines[1:]}

    for i in tumor_labels: 
        if tumor_labels[i] == 'no_tumor': 
            tumor_labels[i] = 0
        elif tumor_labels[i] == "meningioma_tumor":
            tumor_labels[i] = 1
        elif tumor_labels[i] == "glioma_tumor":
            tumor_labels[i] = 2
        elif tumor_labels[i] == "pituitary_tumor":
            tumor_labels[i] = 3
    return(tumor_labels)

def extract_features_with_conv_testset():
    all_features = []
    all_labels = []
    
    labels = categorical_labelling_testset()

    image_paths = [os.path.join(test_images_dir, l) for l in os.listdir(test_images_dir)]
    if os.path.isdir(images_dir):
        all_features = []
        all_labels = []
        for img_path in image_paths:
            filename = img_path.split('/')[-1]
            features = imread(img_path, as_gray=True)
            all_features.append(features)
            all_labels.append(labels[filename])
    np_features = np.array(all_features)
    np_labels = np.array(all_labels)
    
    np_features = np_features.reshape(-1,512, 512,1)
    
    return np_features, np_labels

def get_test_data(): 

    X, Y = extract_features_with_conv_testset()
    
    te_X = X
    te_Y = Y
    
    return te_X, te_Y



## Create CNN


In [8]:
#Assign features and labels to the corresponding variables
x_train, y_train, x_test, y_test = get_data()



In [21]:
# The model is set to be sequential()
model_CNN_plus = Sequential()
model_CNN_plus.add(Conv2D(32, (3, 3), input_shape=(512, 512, 1))) #Setting the first convolution layer and giving the input shape
model_CNN_plus.add(Activation('relu')) # Selecting the activation function linked to the conv layer
model_CNN_plus.add(MaxPooling2D(pool_size=(2, 2))) #Setting the maxpooling layer 

# We add another set of Conv, Activation and MaxPooling to make the CNN stronger. 
model_CNN_plus.add(Conv2D(32, (3, 3)))
model_CNN_plus.add(Activation('relu'))
model_CNN_plus.add(MaxPooling2D(pool_size=(2, 2)))

# We add another set of Conv, Activation and MaxPooling to make the CNN stronger. 
model_CNN_plus.add(Conv2D(64, (3, 3)))
model_CNN_plus.add(Activation('relu'))
model_CNN_plus.add(MaxPooling2D(pool_size=(2, 2)))

model_CNN_plus.add(Flatten())  # This line is used to convert our model from 3d to 1d

#Add the fully connected layer
model_CNN_plus.add(Dense(64))
model_CNN_plus.add(Activation('relu'))
model_CNN_plus.add(Dropout(0.5)) #We use dropout to avoid overfitting
model_CNN_plus.add(Dense(4)) #As we have 4 outputs (3 types of tumour and 1 no tumour), we put 4 categories
model_CNN_plus.add(Activation('sigmoid'))

#Shows the construction of our model (interesting for troubleshooting )
model_CNN_plus.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 510, 510, 32)      320       
_________________________________________________________________
activation_22 (Activation)   (None, 510, 510, 32)      0         
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 255, 255, 32)      0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 253, 253, 32)      9248      
_________________________________________________________________
activation_23 (Activation)   (None, 253, 253, 32)      0         
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 126, 126, 32)      0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 124, 124, 64)     

### Compilation of the model


In [20]:
# optimizer = Adam(lr = 10e-5)
# For compilation we use categorical cross entropy as we have 4 categories. We also use accuracy as metrics and adam optimsier 
model_CNN_plus.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer="adam")



### One-hot-encoding of inputs

In [13]:
# As we have multiple categories, we need to use one-hot-encoding on the labels in order to fit our model use the extracted label and features
y_train = tf.keras.utils.to_categorical(y_train, num_classes=4)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=4)

In [22]:
# The shape of our feature vector is: 
x_train.shape

(2400, 512, 512, 1)

### Training of the model
Note: fitting this model takes more than 10hours to run on a CPU. 

In [23]:
# Training is started here using the x_train and y_train values and validated on x_test and y_test (validation variables)
model_CNN_plus.fit(x_train, y_train,batch_size = 64, epochs = 20 ,validation_data=(x_test, y_test))


Train on 2400 samples, validate on 600 samples
Epoch 1/20
2400/2400 [==============================] - 1160s 483ms/step - loss: 0.5577 - accuracy: 0.7395 - val_loss: 0.4927 - val_accuracy: 0.7571
Epoch 2/20
2400/2400 [==============================] - 1219s 508ms/step - loss: 0.4853 - accuracy: 0.7716 - val_loss: 0.4290 - val_accuracy: 0.8037
Epoch 3/20
2400/2400 [==============================] - 1282s 534ms/step - loss: 0.4223 - accuracy: 0.8018 - val_loss: 0.3791 - val_accuracy: 0.8525
Epoch 4/20
2400/2400 [==============================] - 1277s 532ms/step - loss: 0.3773 - accuracy: 0.8326 - val_loss: 0.3397 - val_accuracy: 0.8671
Epoch 5/20
2400/2400 [==============================] - 1342s 559ms/step - loss: 0.3340 - accuracy: 0.8514 - val_loss: 0.3274 - val_accuracy: 0.8729
Epoch 6/20
2400/2400 [==============================] - 1347s 561ms/step - loss: 0.3115 - accuracy: 0.8652 - val_loss: 0.2982 - val_accuracy: 0.8804
Epoch 7/20
2400/2400 [==============================] - 139

## Saving the model

In [24]:
# Model takes 10 hours to run...better to save it
model_CNN_plus.save('CNN_Model_Cat_one_hot')

## Loading One-Hot Encoded model

In [6]:
# Useful if model is saved before
model = tf.keras.models.load_model('CNN_Model_Cat_one_hot')

2022-01-05 12:41:53.609804: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-05 12:41:53.610120: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


## Evaluating using testing dataset

In [7]:
#We use the functions defined earlier to assign the testing values
x_TEST, y_TEST = get_test_data()
y_TEST = tf.keras.utils.to_categorical(y_TEST, num_classes=4)

y_TEST.shape

In [10]:
result_one_hot = model.evaluate(x_TEST, y_TEST)

print("test loss, test acc:", result_one_hot)

200/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [31]:
# We use the model on testing set
pred = model.predict_classes(x_TEST, verbose = 0)

In [38]:
rounded_testing_labels = np.argmax(y_TEST, axis = 1) #We need to decode the one-hot-encoded output before observing the values
print(rounded_testing_labels)

[1 2 3 1 1 0 1 1 1 1 0 1 2 0 1 0 1 1 3 2 0 2 1 1 1 0 1 1 0 3 1 1 0 3 1 3 2
 2 1 2 0 2 3 2 0 1 3 0 3 1 1 2 3 3 1 2 0 3 3 2 3 3 1 2 3 3 1 0 3 2 1 1 3 1
 3 2 3 2 3 0 2 1 3 3 0 1 3 1 0 1 2 2 2 0 3 2 1 1 1 3 0 3 0 1 3 2 1 0 1 3 1
 1 0 3 2 3 2 2 0 3 2 3 0 2 1 0 3 2 3 3 1 3 1 1 1 0 1 0 1 1 1 2 0 1 0 2 3 2
 1 0 1 1 3 3 0 0 2 3 0 1 3 3 3 2 1 2 0 1 2 1 2 0 0 3 3 1 2 2 1 1 1 1 3 0 0
 2 1 1 2 2 3 2 1 3 2 3 1 1 3 3]


In [40]:
c_report_testing = classification_report(rounded_testing_labels, pred, target_names = ["no tumour", "meningioma_tumor", "glioma_tumor", "pituitary_tumor"])
print(c_report_testing)

                  precision    recall  f1-score   support

       no tumour       0.88      0.81      0.85        37
meningioma_tumor       0.85      0.82      0.84        68
    glioma_tumor       0.88      0.86      0.87        43
 pituitary_tumor       0.90      1.00      0.95        52

        accuracy                           0.88       200
       macro avg       0.88      0.87      0.87       200
    weighted avg       0.87      0.88      0.87       200

